In [65]:
from pyln.client import Gossmap, GossmapNodeId, GossmapNode, GossmapStats

gossmap = Gossmap('gossip_stores/nov_17.gossip_store')

In [4]:
channel = gossmap.get_channel('803775x1091x1')
channel.satoshis

2000000

In [7]:
channel.features

0

In [8]:
channel.half_channels[0].source

GossmapNode[022bd0aa893db4ac890e457cca8c83f112518d6941bf9153dab4bf904620503a78, "lnbits.com                      "]

In [13]:
channel.half_channels[0].destination.node_id

GossmapNodeId[02d503276074ce65fd3aefd220e2fdbd272a9fc0bd23c32f529a53a9c98022abe4]

In [10]:
gossmap.channels

{ShortChannelId[761920x1276x0]: GossmapChannel[761920x1276x0],
 ShortChannelId[702790x539x1]: GossmapChannel[702790x539x1],
 ShortChannelId[721770x2518x0]: GossmapChannel[721770x2518x0],
 ShortChannelId[785923x1933x10]: GossmapChannel[785923x1933x10],
 ShortChannelId[793907x4235x1]: GossmapChannel[793907x4235x1],
 ShortChannelId[771083x1469x1]: GossmapChannel[771083x1469x1],
 ShortChannelId[768170x732x0]: GossmapChannel[768170x732x0],
 ShortChannelId[770873x2090x0]: GossmapChannel[770873x2090x0],
 ShortChannelId[799842x1310x5]: GossmapChannel[799842x1310x5],
 ShortChannelId[785910x1149x12]: GossmapChannel[785910x1149x12],
 ShortChannelId[799842x1310x3]: GossmapChannel[799842x1310x3],
 ShortChannelId[769299x344x27]: GossmapChannel[769299x344x27],
 ShortChannelId[769306x236x0]: GossmapChannel[769306x236x0],
 ShortChannelId[769220x1924x1]: GossmapChannel[769220x1924x1],
 ShortChannelId[785923x1933x15]: GossmapChannel[785923x1933x15],
 ShortChannelId[692080x1833x0]: GossmapChannel[692080x1

In [15]:
gossmap.get_neighbors_hc(destination=channel.half_channels[0].destination.node_id)

{GossmapHalfchannel[803773x2518x0/0],
 GossmapHalfchannel[803773x2518x2/1],
 GossmapHalfchannel[803775x1091x0/1],
 GossmapHalfchannel[803775x1091x1/0],
 GossmapHalfchannel[804114x1166x0/1],
 GossmapHalfchannel[805073x2021x1/1],
 GossmapHalfchannel[805073x2021x2/0],
 GossmapHalfchannel[805073x2021x3/0],
 GossmapHalfchannel[808592x2287x6/1],
 GossmapHalfchannel[809266x2756x0/0],
 GossmapHalfchannel[809277x524x0/1]}

In [16]:
gossmap.get_neighbors_hc(source=channel.half_channels[0].destination.node_id)

{GossmapHalfchannel[803773x2518x0/1],
 GossmapHalfchannel[803773x2518x2/0],
 GossmapHalfchannel[803775x1091x0/0],
 GossmapHalfchannel[803775x1091x1/1],
 GossmapHalfchannel[804114x1166x0/0],
 GossmapHalfchannel[805073x2021x1/0],
 GossmapHalfchannel[805073x2021x2/1],
 GossmapHalfchannel[805073x2021x3/1],
 GossmapHalfchannel[808592x2287x6/0],
 GossmapHalfchannel[809266x2756x0/1],
 GossmapHalfchannel[809277x524x0/0]}

In [17]:
def find_neighbors(gossmap, node, depth=0):
    neighbors = gossmap.get_neighbors(node, depth=depth)
    return neighbors

In [58]:
path = find_route(gossmap, my_source, my_destination) 

depth: 0
depth: 1
path_node is: 039e3d2caa3676f4e5eed072ecd429e02b11e6ebc1db6c95e027aaa753cafe6b35


AssertionError: source or destination unknown

In [63]:
# challenge: what channels should i use to get 
# from 0202e7727ee46d943a44c7bf0dce14220a839630dee67d03446bf935178b55a0df
# to   02c9477078f21e6f17816257132678d2d27f4c7074c8b0114d8be5ce9cee14bc26

node_from = '0202e7727ee46d943a44c7bf0dce14220a839630dee67d03446bf935178b55a0df'
node_to = '02c9477078f21e6f17816257132678d2d27f4c7074c8b0114d8be5ce9cee14bc26'

def find_necessary_depth(gossmap, source, destination):
    intersect = {}
    depth = 0
    while len(intersect) == 0:
        neighbors_source = find_neighbors(gossmap, source, depth=depth)
        neighbors_destination = find_neighbors(gossmap, destination, depth=depth)
        intersect = neighbors_source.intersection(neighbors_destination)
        print(f"depth: {depth}")
        depth += 1
    return depth


def find_route(gossmap, source, destination):
    max_depth = find_necessary_depth(gossmap, source, destination)
    return find_route_inner(gossmap, source, destination, depth=max_depth)
    

def find_route_inner(gossmap, source, destination, depth):
    if are_neighbors(gossmap, source, destination):
        print(f'path of neighbors: {source} {destination}')
        return [source, destination]
    neighbors_source = find_neighbors(gossmap, source, depth)
    neighbors_destination = find_neighbors(gossmap, destination, depth)
    path_node = list(neighbors_source.intersection(neighbors_destination))[0]
    print(f"path_node is: {path_node}")
    print(f"path_node type is: {type(path_node)}")
    left_path = find_route_inner(gossmap, source, path_node, depth)
    print(f"left_node is: {left_node}")
    right_path = find_route_inner(gossmap, path_node, destination, depth)
    print(f"right_node is: {right_node}")
    print(f'combo path: {left_path} {right_path}')
    return left_path[:-1] + right_path
    
destination
def are_neighbors(gossmap, source, destination):
    neighbors_source = find_neighbors(gossmap, source, 0)
    neighbors_destination = find_neighbors(gossmap, destination, 0)
    path_node = neighbors_source.intersection(neighbors_destination)
    return bool(path_node)   

my_source = GossmapNodeId("0202e7727ee46d943a44c7bf0dce14220a839630dee67d03446bf935178b55a0df")
my_destination = GossmapNodeId("02c9477078f21e6f17816257132678d2d27f4c7074c8b0114d8be5ce9cee14bc26")

In [64]:
path = find_route(gossmap, my_source, my_destination)

depth: 0
depth: 1
path_node is: 039e3d2caa3676f4e5eed072ecd429e02b11e6ebc1db6c95e027aaa753cafe6b35
path_node type is: <class 'pyln.client.gossmap.GossmapNode'>


AssertionError: source or destination unknown

In [61]:
path

NameError: name 'path' is not defined

In [59]:
type(3)

int